In [32]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time

In [33]:
money : float = 500000 # 所持金
unit = 10000  # 取引通貨単位(暫定)
ask_val = []  # 取引通貨価格
ask_num = []  # 取引通貨単位
ask_costmoney = []  # 支払い金額
ask_money = [ask_val ,ask_num, ask_costmoney]  # 所持している買い通貨(構造体)
bid_val = []  # 取引通貨価格
bid_num = []  # 取引通貨単位
bid_costmoney = []  # 支払い金額
bid_money = [bid_val ,bid_num, bid_costmoney]  # 所持している売り通貨(構造体)
money_capi = 0  # 保有中のポジションの金額
reva = 25  # レバレッジ
max_epi = 200  # エピソードのMAX
ask_rewards = []  # エピソード毎の報酬リスト
bid_rewards = []  # エピソード毎の報酬リスト
ask_grads = []
bid_grads = []
not_act = 0  # 何もしない
ask_act = 1  # 買い
bid_act = 2  # 売り
act_order_type = [not_act, ask_act, bid_act]  # 注文時
act_settlement_bid_type = [not_act, bid_act]  # 買い注文決済時
act_settlement_ask_type = [not_act, ask_act]  # 売り注文決済時
end_epi = True  # エピソード終了フラグ
count_epi = 0  # エピソードカウンター
X_column = ['日付', 'ドル円', 'ポンド円', 'オシレーター買いドル円', 'オシレーター中立ドル円', 'オシレーター売りドル円',
       '移動平均買いドル円', '移動平均中立ドル円', '移動平均売りドル円', 'オシレーター買いポンド円', 'オシレーター中立ポンド円',
       'オシレーター売りポンド円', '移動平均買いポンド円', '移動平均中立ポンド円', '移動平均売りポンド円', 'S&P500指数',
       'S&P500指数変化率', '日経平均株価', '日経平均株価変化率', 'UK100INDEX', 'UK100INDEX変化率',
       '恐怖指数', '恐怖指数変化率']

In [34]:
def data_shape():
    #------------csvからゴミデータを削除---------------------------
    df= pd.read_csv("為替情報.csv",encoding='shift_jis',index_col=0)
    #条件にマッチしたIndexを取得
    drop_index = df.index[df['ドル円(15分後)'] == 0]
    #条件にマッチしたIndexを削除
    df = df.drop(drop_index)
    df = df.dropna()  # NaNを削除
    df.to_csv('FX再現用pd自動整形.csv', encoding='shift_jis')
    #------------csvからゴミデータを削除---------------------------

    df= pd.read_csv("FX再現用pd自動整形.csv",encoding='shift_jis')
    print(df)
    X = df[X_column]  # 特徴量
    y = df['ドル円(15分後)']  # 目的変数
    print(df.shape)
    print('特徴量：',X)
    print('目的変数：',y)
    #usdjpy = df.at[count_epi, 'ドル円']
    #********為替エピソード******************
    return df, X, y

def act_new_bid(money, usdjpy, acted_flag):  # エピソード毎の行動　　注文(買い、売り)するか、しないか
    random_act = random.choice(act_order_type)  # 行動

    if random_act == not_act or random_act == ask_act:
        return  str('新規注文：無し'), money, acted_flag

    elif random_act == bid_act:
        biding_money = usdjpy*unit / 25 # 消費金額
        money = money - biding_money  # 注文後の所持金 = 所持金　- 注文価格
        biding_money = round(biding_money, 3)  # 少数第３位まで表示
        money = round(money, 3)  # 少数第３位まで表示

        bid_val.append(usdjpy) # 取引価格
        bid_num.append(unit)  # 取引単位
        bid_costmoney.append(biding_money)  # 現在の所持金
        acted_flag = True  # 注文したとき新規注文行動フラグをTrueにする
        return str('新規注文：売り'), money, acted_flag

def ask_settlement(money, usdjpy):  # 売りポジションを保有している時の行動　決済(買い)するか、しないか)
    benefit = 0
    if bid_val:  # 売り注文リストに要素が入っているか

        # 売り注文リストに要素が入っている場合
        random_act = random.choice(act_settlement_ask_type)

        if random_act == not_act:  # 1/2で何もしない
            ask_acted_num = 1
            return str('無し'), money, benefit, ask_acted_num

        if random_act == ask_act:  # 1/2で保有している中で1番目の要素を決済する
            diff = bid_val[0] - usdjpy  # 現在の価格と保有しているポジションの差額を求める
            benefit = diff * bid_num[0]  # 儲け = 差額 *保有数　
            benefit = round(benefit, 3)  # 少数第３位まで表示
            money = money + benefit + bid_costmoney[0] # 所持金額に反映
            del bid_val[0]
            del bid_num[0]
            del bid_costmoney[0]
            ask_acted_num = 9
            return str('買い'), money, benefit, ask_acted_num

    ask_acted_num = 0
    return str('保有無し'), money, benefit, ask_acted_num  # 売り注文リストに要素が入っていない


In [35]:
def bid_ask_settlement(end_epi, count_epi, money, money_capi, ask_rewards):  # 1エピソード行う
    acted_flag = False  # 新規注文行動フラグ
    while end_epi:
            usdjpy = df.at[count_epi, 'ドル円']

            if acted_flag == False:  # 新規注文行動フラグがFalseのとき、新規決済行動を行う
                order_acted, money, acted_flag = act_new_bid(money, usdjpy, acted_flag) # 新規の行動(買う、売る、何もしない)

            settlement_ask_acted, money, ask_benefit, ask_acted_num = ask_settlement(money, usdjpy)
            ask_rewards.append(ask_benefit)
            print(count_epi+1,'step目:売り注文に対する決済', settlement_ask_acted,'利益', ask_benefit ,'決済後所持金:',str(money),'円')

            if ask_acted_num == 9:  # 買い決済の時終了する
                print('***1エピソード終了(売り⇒買い)***')
                end_epi = False

            for i in zip(ask_val, ask_num, ask_costmoney):  # 保有中のポジションの価格
                money_capi += usdjpy * int(i[1]) / reva
            for i in zip(bid_val, bid_num, bid_costmoney):  # 保有中のポジションの価格
                money_capi += usdjpy * int(i[1]) / reva
            money_capi += money  # 時価総額

            print(count_epi+1,'step目:新規注文', order_acted,'所持金:',str(money),'円\n',
                 '買い注文:\n',ask_val,'\n',ask_num,'\n',ask_costmoney,'\n',
                 '売り注文:\n',bid_val, '\n',bid_num,'\n',bid_costmoney)
            print('手持ちの時価価額(現金+ポジション)：', money_capi,"""'総報酬：',reward,'\n'""")

            count_epi = count_epi + 1  # エピソードの最後にカウンタを増やす
            money_capi = 0  # 時価総額を初期化する
    return ask_benefit, count_epi  # return：報酬,

In [36]:
df, X, y = data_shape()  # データ整形

                        日付      ドル円     ポンド円  オシレーター買いドル円  オシレーター中立ドル円  \
0     2020/12/17/ 06:24:11  103.481  139.650          2.0          7.0   
1     2020/12/17/ 06:25:45  103.483  139.661          2.0          7.0   
2     2020/12/17/ 06:27:20  103.480  139.698          2.0          7.0   
3     2020/12/17/ 06:28:54  103.470  139.674          2.0          7.0   
4     2020/12/17/ 06:30:29  103.474  139.683          2.0          7.0   
...                    ...      ...      ...          ...          ...   
1960  2020/12/19/ 10:04:17  103.275  139.783          2.0          6.0   
1961  2020/12/19/ 10:05:51  103.275  139.783          2.0          6.0   
1962  2020/12/19/ 10:07:25  103.275  139.783          2.0          6.0   
1963  2020/12/19/ 10:08:59  103.275  139.783          2.0          6.0   
1964  2020/12/19/ 10:10:34  103.275  139.783          2.0          6.0   

      オシレーター売りドル円  移動平均買いドル円  移動平均中立ドル円  移動平均売りドル円  オシレーター買いポンド円  ...  \
0             2.0        0.0        1.

In [40]:
iteration = 10  # 1イテレーションは10エピソード

for i in range(iteration):  # iイテレーション
    ask_benefit, count_epi = bid_ask_settlement(end_epi, count_epi, money, money_capi, ask_rewards)  # 1エピソード行う
    print('ask_rewards：',ask_rewards)
X_train = X[:count_epi]
print(X_train.shape)
print(X.shape)

2 step目:売り注文に対する決済 無し 利益 0 決済後所持金: 458606.8 円
2 step目:新規注文 新規注文：売り 所持金: 458606.8 円
 買い注文:
 [] 
 [] 
 [] 
 売り注文:
 [103.483] 
 [10000] 
 [41393.2]
手持ちの時価価額(現金+ポジション)： 500000.0 '総報酬：',reward,'
'
3 step目:売り注文に対する決済 買い 利益 30.0 決済後所持金: 500030.0 円
***1エピソード終了(売り⇒買い)***
3 step目:新規注文 新規注文：売り 所持金: 500030.0 円
 買い注文:
 [] 
 [] 
 [] 
 売り注文:
 [] 
 [] 
 []
手持ちの時価価額(現金+ポジション)： 500030.0 '総報酬：',reward,'
'
ask_rewards： [0.0, 0, 30.0]
4 step目:売り注文に対する決済 保有無し 利益 0 決済後所持金: 500000 円
4 step目:新規注文 新規注文：無し 所持金: 500000 円
 買い注文:
 [] 
 [] 
 [] 
 売り注文:
 [] 
 [] 
 []
手持ちの時価価額(現金+ポジション)： 500000 '総報酬：',reward,'
'
5 step目:売り注文に対する決済 保有無し 利益 0 決済後所持金: 500000 円
5 step目:新規注文 新規注文：無し 所持金: 500000 円
 買い注文:
 [] 
 [] 
 [] 
 売り注文:
 [] 
 [] 
 []
手持ちの時価価額(現金+ポジション)： 500000 '総報酬：',reward,'
'
6 step目:売り注文に対する決済 無し 利益 0 決済後所持金: 458613.2 円
6 step目:新規注文 新規注文：売り 所持金: 458613.2 円
 買い注文:
 [] 
 [] 
 [] 
 売り注文:
 [103.467] 
 [10000] 
 [41386.8]
手持ちの時価価額(現金+ポジション)： 500000.0 '総報酬：',reward,'
'
7 step目:売り注文に対する決済 無し 利益 0 決済後所持金: 458613.2 円
7 step